In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/melbourne-housing-snapshot/melb_data.csv


This is an example for dealing with categorical values and columns with Null values
The data set is used of Melbourne Housing Snapshot

In [2]:
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

In [3]:
data.head()

,Unnamed: 0,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,5,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,6,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [4]:
from sklearn.model_selection import train_test_split

y = data.Price
melb_predictors = data.drop(['Price', 'Unnamed: 0'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])
#X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

In [5]:
X.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
0,2,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,-37.7996,144.9984,4019.0
1,2,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,-37.8079,144.9934,4019.0
2,3,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,-37.8093,144.9944,4019.0
3,3,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,-37.7969,144.9969,4019.0
4,4,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,-37.8072,144.9941,4019.0


Splitting data into training (X_train) and validation (X_valid)

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

Model and MAE score calculation method

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

Dealing with missing values using simple imputation technique (simple_imputer) other than dropping columns with missing values. 

In [8]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach (Imputation: SimpleImputer()):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach (Imputation: SimpleImputer()):
198075.20472666025


In [9]:
imputed_X_train.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
0,4.0,7.8,3058.0,4.0,2.0,1.0,381.0,152.120627,1938.00000,-37.7337,144.9548,11204.0
1,3.0,7.8,3124.0,3.0,1.0,1.0,544.0,160.000000,1930.00000,-37.8436,145.0581,8920.0
2,2.0,5.6,3101.0,2.0,1.0,1.0,121.0,152.120627,1966.09975,-37.8126,145.0534,10331.0
3,3.0,7.5,3123.0,3.0,2.0,2.0,200.0,152.120627,1966.09975,-37.8396,145.0514,6482.0
4,2.0,4.5,3181.0,2.0,1.0,1.0,2842.0,84.000000,1920.00000,-37.8513,144.9943,7717.0


The above data is imputed data set means the null values have dealt in the above data set. The following is the size of the imputed data set and the oringinal data set compared. The object type columns were excluded in imputation

In [10]:
print("Imputed data for null values :", imputed_X_train.shape)
print("Original data with null values :", melb_predictors.shape)

Imputed data for null values : (14716, 12)
Original data with null values : (18396, 20)


The figures show that exactly 8 columns were excluded from the original data set. Now we will deal the categorical columns which are included in ojbect type columns. 

In [11]:
melb_predictors.head()

,Suburb,Address,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [12]:
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

y = data.Price
X = data.drop(['Price', 'Unnamed: 0'], axis=1)

In [13]:
X.head()

,Suburb,Address,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [14]:
# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)


In [15]:
print("X shape of data :", X.shape)
print("X_train_full shape of data :", X_train_full.shape)
X_train_full.head()

X shape of data : (18396, 20)
X_train_full shape of data : (14716, 20)


,Suburb,Address,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
2573,Coburg,45 Kerferd St,4,h,SP,Peter,17/09/2016,7.8,3058.0,4.0,2.0,1.0,381.0,NaN,1938.0,Moreland,-37.7337,144.9548,Northern Metropolitan,11204.0
2091,Camberwell,13 Grace St,3,h,SP,Noel,12/11/2016,7.8,3124.0,3.0,1.0,1.0,544.0,160.0,1930.0,Boroondara,-37.8436,145.0581,Southern Metropolitan,8920.0
4683,Kew,3/40 Rowland St,2,u,S,Jellis,28/05/2016,5.6,3101.0,2.0,1.0,1.0,121.0,NaN,NaN,Boroondara,-37.8126,145.0534,Southern Metropolitan,10331.0
8832,Hawthorn East,2/2 Myrniong Gr,3,h,VB,Marshall,7/05/2016,7.5,3123.0,3.0,2.0,2.0,200.0,NaN,NaN,Boroondara,-37.8396,145.0514,Southern Metropolitan,6482.0
10469,Prahran,303/10 Hillingdon Pl,2,u,S,Gary,13/05/2017,4.5,3181.0,2.0,1.0,1.0,2842.0,84.0,1920.0,Stonnington,-37.8513,144.9943,Southern Metropolitan,7717.0


Dealing with categorical columns in the X_train_full data set. The X_train_full data set is the portion of the original data X. Here we will drop the columns with missing values. 

In [16]:
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()]
#X_train_full.drop(cols_with_missing, axis=1, inplace=True)
#X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

print("Columns with missing values :", cols_with_missing)
print("X_train_ful shape :", X_train_full.shape)

Columns with missing values : ['Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude', 'Longtitude', 'Regionname', 'Propertycount']
X_train_ful shape : (14716, 20)


Recognizing the Categorical columns

In [17]:
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

print('low cardinality columns :', low_cardinality_cols)
print('numerical columns :', numerical_cols)


low cardinality columns : ['Type', 'Method', 'Regionname']
numerical columns : ['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount']


At this stage we know the low_cardinality columns (categorical columns) and the numerical columns we now know make another data set that will include both the categorical columns and and numerical columns. 

In [18]:
my_cols = low_cardinality_cols + numerical_cols
print('My cols :', my_cols)
X_train_full.head()

My cols : ['Type', 'Method', 'Regionname', 'Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount']


,Suburb,Address,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
2573,Coburg,45 Kerferd St,4,h,SP,Peter,17/09/2016,7.8,3058.0,4.0,2.0,1.0,381.0,NaN,1938.0,Moreland,-37.7337,144.9548,Northern Metropolitan,11204.0
2091,Camberwell,13 Grace St,3,h,SP,Noel,12/11/2016,7.8,3124.0,3.0,1.0,1.0,544.0,160.0,1930.0,Boroondara,-37.8436,145.0581,Southern Metropolitan,8920.0
4683,Kew,3/40 Rowland St,2,u,S,Jellis,28/05/2016,5.6,3101.0,2.0,1.0,1.0,121.0,NaN,NaN,Boroondara,-37.8126,145.0534,Southern Metropolitan,10331.0
8832,Hawthorn East,2/2 Myrniong Gr,3,h,VB,Marshall,7/05/2016,7.5,3123.0,3.0,2.0,2.0,200.0,NaN,NaN,Boroondara,-37.8396,145.0514,Southern Metropolitan,6482.0
10469,Prahran,303/10 Hillingdon Pl,2,u,S,Gary,13/05/2017,4.5,3181.0,2.0,1.0,1.0,2842.0,84.0,1920.0,Stonnington,-37.8513,144.9943,Southern Metropolitan,7717.0


In [19]:
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [20]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
2573,h,SP,Northern Metropolitan,4,7.8,3058.0,4.0,2.0,1.0,381.0,NaN,1938.0,-37.7337,144.9548,11204.0
2091,h,SP,Southern Metropolitan,3,7.8,3124.0,3.0,1.0,1.0,544.0,160.0,1930.0,-37.8436,145.0581,8920.0
4683,u,S,Southern Metropolitan,2,5.6,3101.0,2.0,1.0,1.0,121.0,NaN,NaN,-37.8126,145.0534,10331.0
8832,h,VB,Southern Metropolitan,3,7.5,3123.0,3.0,2.0,2.0,200.0,NaN,NaN,-37.8396,145.0514,6482.0
10469,u,S,Southern Metropolitan,2,4.5,3181.0,2.0,1.0,1.0,2842.0,84.0,1920.0,-37.8513,144.9943,7717.0


Now we will apply the technique on OneHotEncoder for setting up categorical columns into numerical columns

In [21]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Type', 'Method', 'Regionname']


In [22]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)


In [23]:
print("Shape X_train :", X_train.shape)
print("Shape OH_X_train :", OH_X_train.shape)

Shape X_train : (14716, 15)
Shape OH_X_train : (14716, 29)


In [24]:
OH_X_train.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,...,7,8,9,10,11,12,13,14,15,16
2573,4,7.8,3058.0,4.0,2.0,1.0,381.0,NaN,1938.0,-37.7337,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2091,3,7.8,3124.0,3.0,1.0,1.0,544.0,160.0,1930.0,-37.8436,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4683,2,5.6,3101.0,2.0,1.0,1.0,121.0,NaN,NaN,-37.8126,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8832,3,7.5,3123.0,3.0,2.0,2.0,200.0,NaN,NaN,-37.8396,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10469,2,4.5,3181.0,2.0,1.0,1.0,2842.0,84.0,1920.0,-37.8513,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


Checking for null values in OH_X_train (one hot encoded data set dealing categorical columns): 

In [25]:
print("the numer of columns with Null values :", OH_X_train.isnull().sum().sum())

the numer of columns with Null values : 33645


Now we will apply the Simple Imputer technique for dealing with null values in OH_X_train data set. 

In [26]:
# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(OH_X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(OH_X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = OH_X_valid.columns
imputed_X_valid.columns = OH_X_valid.columns



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [27]:
imputed_X_train.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,...,7,8,9,10,11,12,13,14,15,16
0,4.0,7.8,3058.0,4.0,2.0,1.0,381.0,152.120627,1938.00000,-37.7337,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,7.8,3124.0,3.0,1.0,1.0,544.0,160.000000,1930.00000,-37.8436,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2.0,5.6,3101.0,2.0,1.0,1.0,121.0,152.120627,1966.09975,-37.8126,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3.0,7.5,3123.0,3.0,2.0,2.0,200.0,152.120627,1966.09975,-37.8396,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2.0,4.5,3181.0,2.0,1.0,1.0,2842.0,84.000000,1920.00000,-37.8513,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [28]:
print("MAE from Approach (Imputation: SimpleImputer()):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach (Imputation: SimpleImputer()):


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


183564.16651003467


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
